# Estrazione Nomi

## Da pdf o txt

In [2]:
from __future__ import print_function
from __future__ import division
import os
import re
import textract
import lib.words as wd
import lib.text_extraction as te
import glob
from lib.utils import uniq_list

In [2]:
def join_cognomi_articles(words):
    articles = set([u'de', u'di', u'du', u'del', u'dell', u'la', u'dei', u'le', 
                    u'della', u'dall', u'dalla', u'dello', 'degli', 'lo', 'art',
                   u'dal', u'dalle'])
    found = False
    joined_words = []
    for word in words:
        if not found and word.lower() in articles:
            found = True
            cur_word = word
        elif found:
            cur_word += ' '+word
            joined_words.append(cur_word)
            found = False
        else:
            joined_words.append(word)
    return joined_words

In [3]:
#nomi = list(n.strip().lower() for n in open('../extraction/nomi/nomi_asti.txt'))
#nomi = set(n.strip().lower() for n in open('../extraction/nomi/nomi_gnorry.txt'))
nomi = set(n.strip().lower() for n in open('../extraction/nomi/nomi_italiani.txt'))
notais = set(n.strip().lower() for n in open('../extraction/nomi/notari.txt'))
#cognomi = set(n.strip().lower() for n in open('../extraction/nomi/cognomi.txt'))
cognomi = set(n.strip().lower() for n in open('../extraction/nomi/cognomi_big.txt'))
nomi_cognomi_list = [w for n in open('../extraction/nomi/notari.txt') for w in n.strip().lower().split(' ')]
nomi_cognomi_notai = set(join_cognomi_articles(nomi_cognomi_list))

In [4]:
nomi_cognomi_notai

{'riccardi',
 'mancioppi',
 'riccardo',
 'di pierdomenico',
 'perrini',
 'insolia',
 'fugazzola',
 'bottino',
 'caiazzo',
 'cipolletti',
 'manfredini',
 'bottini',
 'francesco',
 'francesca',
 'gabriella',
 'porqueddu',
 'manfredi',
 'lenoci',
 'antognazza',
 'mazzarella',
 'iorio',
 'puggioni',
 'guerreri',
 'di marino',
 'gualdrini',
 'scola',
 'pantano',
 'ebner',
 'pantani',
 'ciambrelli',
 'marcoz',
 'patalano',
 'sorrentini',
 'lori',
 'chicco',
 'gianantonio',
 'gennarini',
 'carapelle',
 'giampietro',
 'orefice',
 'sfera',
 'mottola',
 'rigano',
 'capua',
 'silocchi',
 'previti',
 'petralia',
 'gigliola',
 'perrino',
 'cisotto',
 'faucci',
 'pensato',
 'porto',
 'improta',
 'rosalba',
 'dorina',
 'casas',
 'bertolucci',
 'baroni',
 'fiengo',
 'fiorito',
 'de dominicis',
 'rossetti',
 'cianci',
 'ricciotti',
 'la ciura',
 'stirpe',
 'bonaca',
 'andretta',
 'giordana',
 'girolamo',
 'piva',
 'falce',
 'paleari',
 'colantoni',
 'pescatori',
 'vincenzi',
 'la cava',
 'cascone',
 'v

In [3]:
nomi = nomi[:200]
v = [r.split('\t')[0] for r in nomi]

In [5]:
v += ['ambrogio', 'paolina', 'irma']

In [3]:
def extract_notaio_from_doc(dirr, is_pdf=False, max=-1):
    text = ""
    for i,f in enumerate(os.listdir(dirr)):
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            text = textract.process(dirr + f).strip()
        else: 
            with open (dirr + f, 'r') as f2:
                text = f2.read()
        text = wd.replace_evil_dots_and_underscores(text, rep=r' ').replace('\n',' ').replace(',','')
        text = re.sub('\s+', ' ', text).strip()
        found = False
        for i,line in enumerate(wd.split_sents(text)):
            if found:
                break
            l = line.strip()
            if l != '':
                words = l.split(' ')
                for word in words:
                    if word.lower() == 'me' or word.lower() == 'sottoscritto':
                        index = words.index(word)
                        print(f, ' '.join(words[index+1:index+6]))
                        found = True
                        break
        if not found:
            print('NON TROVATO', '\t\t', f)

In [62]:
extract_notaio_from_doc('../extraction/files_to_label/', True)

dott LUCIA UCCELLI notaio
dott Ambrogio Romano Notaio
Dottoressa Paolina De Liguori
dottor GIOVANNI MURARO Notaio
dottor VINCENZO CALDERINI Notaio
dottor FABRIZIO PASCUCCI notaio
Alessandra Bianchi notaio in
Avv Lucia Di Lieto
Dottor Roberto Colucci Notaio
dottor DONATO CASCAVILLA Notaio
dottor Maria Luisa Sperandeo
Andrea Guglielmoni Notaio in
dottor TOMMASO GAETA Notaio
PAOLA TOTARO Notaio in
Giovanna Falco notaio in
dottor Michele Colasanto Notaio
dottor FABRIZIO PASCUCCI Notaio
Avv Giovanni Piacitelli Notaio
Dottor Notaio PAOLO MICHELI
Dottor Marco Cavallini Notaio


In [39]:
extract_notaio_from_doc('../atti_costitutivi_txt/', False, 500)

5049618730001.txt Fabrizio Figurelli Notaio in Cavezzo
4930258720001.txt Dr Raimondo MALINCONICO Notaio in
5003601230001.txt Dott Andrea Coletta Notaio in
5060167590001.txt Dottor LUCIANO QUAGGIA Notaio in
4985610910001.txt dott PIETRO BERNARDI FABBRANI Notaio
4934063060001.txt dottor FILIPPO MARINELLI Notaio in
4943577600001.txt Zaira LO BIANCO Notaio in
5024081290001.txt Dottor ANGELO CACCETTA Notaio in
5005776590001.txt FABIO MONACO notaio in Viareggio
5051922110001.txt dottor Francesco Figlioli notaio residente
4919128160001.txt dottor Giovannibattista Musto Notaio in
5067992100001.txt Dott ssa ORFINA SCROCCO Notaio
5002413830001.txt presso il Dottor MARCO Collegio
5044504790001.txt dott ssa ANNA LAURA PAOLETTI
5052464430001.txt dottor Luigi BARTOLOMEO Notaio in
5121967110001.txt dott Roberto Chiari fu Leopoldo
5090226530001.txt dottor Salvatore Cutrupia notaio in
4923509370001.txt dr Pasquale Salvatore Placida Notaio
4985596990001.txt dottor GIOVANNI MURARO Notaio del
511074992000

## Dalle frasi

In [16]:
def extract_notaio_from_sentences(doc):
    '''
    Estrai notaio da un documento.
    Arguments:
        - doc: il documento (lista di frasi)
    Returns:
        - lista di nomi, lista di cognomi
    '''
    #w.lower() in nomi or w.lower() in cognomi or w.lower() in whitelist \or
    whitelist = ['de', 'di'] #, 'dei', 'del', 'della', 'degli', 'delle', 'da', 'dello']
    
    for frase in doc:
        frase = frase.replace('\n',' ').replace(',','')
        frase = re.sub('\s+', ' ', frase).strip()
        if frase != '':
            words = re.split('\W', frase)
            #frase.split(' ')
            for word in words:
                if word.lower() == 'me' or word.lower() == 'sottoscritto' or word.lower() == 'mir':
                    index = words.index(word)
                    sub = words[index+1:index+6]
                    res = ''
                    for w in sub:
                        if  any((w.lower() in set(x.lower() for x in riga.split(' '))) for riga in notais):
                            res += w + ' '
                        #if any((w.lower() in set(x.lower() for x in riga.split(' '))) for riga in notais):
                        #    res += w + ' '
                    res +=  '\t\t' +' '.join(sub)   
                    return res     
                    
    return ""

In [114]:
for i,f in enumerate(os.listdir('../atti_costitutivi_txt/')):
    if i == 500:
        break
    text = ""    
    with open ('../atti_costitutivi_txt/' + f, 'r') as f2:
        text = f2.read()
    sents = wd.sentences_doc(text, rep=r' ')    
    res = extract_notaio_from_sentences(sents)
    print(i, f, '\t', res)    

0 5049618730001.txt 	 Fabrizio Figurelli 		Fabrizio Figurelli Notaio in Cavezzo
1 4930258720001.txt 	 Raimondo MALINCONICO 		Dr Raimondo MALINCONICO Notaio in
2 5003601230001.txt 	 Andrea Coletta 		Dott Andrea Coletta Notaio in
3 5060167590001.txt 	 LUCIANO QUAGGIA 		Dottor LUCIANO QUAGGIA Notaio in
4 4985610910001.txt 	 PIETRO BERNARDI FABBRANI 		dott PIETRO BERNARDI FABBRANI Notaio
5 4934063060001.txt 	 FILIPPO MARINELLI 		dottor FILIPPO MARINELLI Notaio in
6 4943577600001.txt 	 Zaira LO BIANCO 		Zaira LO BIANCO Notaio in
7 5024081290001.txt 	 ANGELO CACCETTA 		Dottor ANGELO CACCETTA Notaio in
8 5005776590001.txt 	 FABIO MONACO 		FABIO MONACO notaio in Viareggio
9 5051922110001.txt 	 Francesco Figlioli 		dottor Francesco Figlioli notaio residente
10 4919128160001.txt 	 Giovannibattista Musto 		dottor Giovannibattista Musto Notaio in
11 5067992100001.txt 	 ORFINA SCROCCO 		Dott ssa ORFINA SCROCCO Notaio
12 5002413830001.txt 	 MARCO 		presso il Dottor MARCO Collegio
13 5044504790001.tx

##### Ignora

In [28]:
def extract_notaio_from_end(doc):
    whitelist= ['notaio', '(l.s.)', '(segue sigillo)', 'impronta']
    puntegg = ['\n', '.', ',', ';']
    for frase in reversed(doc):
        if frase != '':
            words = wd.splitted_words_utf8(frase)
            for word in words:
                if word.lower() in whitelist:
                    index = words.index(word)
                    begin = -1
                    print(f, '\t', ' '.join(words[:index]))
                    for w in reversed(words[:index]):
                        if any(p in w for p in puntegg): 
                            begin = words.index(w)
                    if begin > -1:
                        print(f, '\t', ' '.join(words[begin:index]))
                        return
                    return
   
    return "NOPE"

In [29]:
for i,f in enumerate(os.listdir('../atti_costitutivi_txt/')):
    if i == 10:
        break
    text = ""    
    with open ('../atti_costitutivi_txt/' + f, 'r') as f2:
        text = f2.read()
    sents = wd.sentences_doc(text, rep=r' ')  
    extract_notaio_from_end(sents)
    #print(i, f, '\t', res)   

5049618730001.txt 	F to Fabrizio Figurelli
4930258720001.txt 	Salerno 16 febbraio 2017 Firmato digitalmente Raimondo Malinconico
5003601230001.txt 	Firmato Dosio Roberta Pallocchini Stefano Usini Giorgio Andrea Coletta
5060167590001.txt 	Firmato Jorillo Hans Firmato Gilardi Roberto Firmato Luciano Quaggia
4985610910001.txt 	F to Pietro Bernardi Fabbrani
4934063060001.txt 	F to VERBICARO Ruben VERBICARO Noemi Sergio CALVANO Filippo MARINELLI
4943577600001.txt 	Firmato Cicala Pasqualino Al Refaie Fares Restuccia Mariaelisa Zaira Lo Bianco
5024081290001.txt 	Firmato Auteri Mamta Angelo Caccetta
5005776590001.txt 	F to Roberta Girolami F to Vinicio Tuccori F to Fabio Monaco
5051922110001.txt 	Bollo assolto ai sensi del Decreto 22 febbraio 2007 mediante M U I Atto registrato telematicamente presso l Agenzia delle Entrate Ufficio Territo riale di Palermo 1 in data 05 maggio 2017 al n 5201 serie 1T Palermo nel mio studio Via Principe di Paternò n 42 Lì diciassette maggio duemiladiciassette Fi

In [5]:
#paolo

def join_apostrophe(words):
    joined = ' '.join(words)
    rep_d = joined.replace(' d ', ' d\'').replace(' D ', ' D\'')
    return rep_d.split()

def extract_notaio_sent(sentence, notaio_names):
    words = join_apostrophe(wd.splitted_words_utf8(sentence))
    words = join_cognomi_articles(words)
    return [word for word in words if word.lower() in notaio_names and word[0].isupper()]

def find_me(words):
    lwords = [w.lower() for w in words]
    im = -1
    arts = set(['me', 'sottoscritto', 'mir'])
    for i, word in enumerate(lwords):
        if any((w in arts) for w in word.split()):
            return i
    return im

def extract_notaio_name(doc, notaio_names, neigh=5, max_names=3):
    words = [w for sent in doc for w in wd.splitted_words_utf8(sent)]
    j_words = join_cognomi_articles(join_apostrophe(words))
    im = find_me(j_words)
    if im >= 0:
        m_words = j_words[im+1:im+neigh]
    else:
        m_words = j_words
    found_names = [word for word in m_words if word.lower() in notaio_names and word[0].isupper()]
    return uniq_list(found_names)[:max_names]
    

In [6]:
for f in glob.glob('../extraction/esempi_descrizioni2/*.pdf'):
    txt = te.extract_text(f)
    sents = wd.sentences_doc(txt, rep=' ')
    name = extract_notaio_name(sents, nomi_cognomi_notai)
    print(f.split('/')[-1], name)

5120119780001.pdf [u'Francesco', u'PENE', u'VIDARI']
5120873050005daxml-3.pdf []
5120831630001.pdf [u'Sergio', u'Graziosi']
IdentificazioneInformazioniAtti-2.pdf []
5119550990001.pdf [u'LUDOVICO', u'MARIA', u'CAPUANO']
5118869500001.pdf [u'AMELIA', u'ANNA']
5118833510001.pdf [u'Paolo', u'Talice']
5122441020001.pdf [u'Irma', u'Barchi']
5119279170001.pdf [u'MARCO', u'PRINCIVALLE']
5118869880001.pdf [u'MARCO', u'RE']


In [6]:
txt_dir = '../atti_costitutivi_txt/'
l = []
for i, f in enumerate(os.listdir(txt_dir)):
    txt = open(txt_dir+f).read()
    sents = wd.sentences_doc(txt, rep=' ')
    name = extract_notaio_name(sents, nomi_cognomi_notai)
    l.append(name)
    #print(i)
    #print(f, ' '.join(name))
    #print(txt)


KeyboardInterrupt: 

In [132]:
100*sum(1 for n in l if len(n)>1)/len(l)

91.8528025273342

In [125]:
len(os.listdir(txt_dir))

40517

In [37]:
join_cognomi_articles(u'ciao della maggio uno del pietro caio'.split())

[u'ciao', u'della maggio', u'uno', u'del pietro', u'caio']

# Estrazione STATUTO

In [33]:
def extract_statuto_from_sentences_1(dirr, is_pdf=False, max=-1):
    begin_statuto = ['STATUTO','FUNZIONAMENTODELLASOCIET']
    text = ""
    inizio = -1
    index = -1
    fine = -1
    for i,f in enumerate(os.listdir(dirr)):
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            text = textract.process(dirr + f).strip()
        sents = wd.sentences_doc(text, rep=r' ')    
        notaio = extract_notaio_from_sentences(sents)  
        print(f)
        print('notaio', notaio, '\n')

        for frase in sents:  
            if any(b in re.sub('\s+', '', frase) for b in begin_statuto):
                #print i, f, '\t', frase, '\n'
                inizio = sents.index(frase)
            if notaio.lower() in frase.lower() and inizio > 0:
                fine = sents.index(frase)
                break
            
        if fine > -1:
            #print sents[inizio][index:]+'\n'+'\n'.join(sents[inizio+1:fine+1]) + '\n\n\n'
            print('\n'.join(sents[inizio:fine+1]) + '\n\n\n')

In [55]:
def extract_statuto(dirr, is_pdf=False, max=-1):
    begin_statuto = ['STATUTO','FUNZIONAMENTODELLASOCIET']
    text = ""
    for i,f in enumerate(os.listdir(dirr)):
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            text = textract.process(dirr + f).strip()
        sents = wd.sentences_doc(text, rep=r' ')    

        for frase in sents:  
            if any(b in re.sub('\s+', '', frase) for b in begin_statuto):
                print(i, f, '\t', frase, '\n')

In [16]:
def extract_statuto_from_sentences(dirr, is_pdf=False, max=-1):
    begin_statuto = ['S T A T U T O', 'STATUTO', 'STATUTO DELLA SOCITET', 'FUNZIONAMENTO DELLA SOCIET', 'PATTI DISCIPLINANTI']
    
    for i,f in enumerate(os.listdir(dirr)):
        text = ""
        inizio = -1
        index = -1
        fine = -1
        if i == 0:
            continue
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            text = textract.process(dirr + f).strip()
        sents = wd.sentences_doc(text, rep=r' ')    
        notaio = extract_notaio_from_sentences(sents)  
        print(f)
        print('notaio', notaio, '\n')
        for k, frase in enumerate(sents):
            if index == -1:
                for b in begin_statuto:
                    j = wd.index_ignore_whitespaces(frase, b)
                    if j > -1:
                        inizio = k
                        index = j
                        break
                
            if notaio.lower() in frase.lower() and inizio > 0:
                fine = sents.index(frase)
                if fine > inizio:
                    break
            
        if fine > -1:
            print(sents[inizio][index:]+'\n'+'\n'.join(sents[inizio+1:fine+1]) + '\n\n\n')

In [ ]:
extract_statuto_from_sentences('../atti_costitutivi/', True, 4)

In [41]:
from shutil import copyfile

def copy_files_with_statuto(dirr, is_pdf=False, max=-1):
    begin_statuto = ['S T A T U T O', 'STATUTO', 'STATUTO DELLA SOCITET', 'FUNZIONAMENTO DELLA SOCIET', 'PATTI DISCIPLINANTI']
    
    for i,f in enumerate(os.listdir(dirr)):
        text = ""
        if i == 0:
            continue
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            try:
                text = textract.process(dirr + f).strip()
            except:
                continue
        sents = wd.sentences_doc(text, rep=r' ')    
        to_move = False
        
        for k, frase in enumerate(sents):
            if frase.strip() != "":
                for b in begin_statuto:
                    try:
                        j = wd.index_ignore_whitespaces(frase, b)
                    except:
                        print("Exception in: "+frase)
                        j = -1
                    if j > -1:
                        inizio = k
                        index = j
                        to_move = True
                        break
                    
        if to_move:            
            copyfile(str(dirr + f),str("../statuti/" + f))
            
        print(i, f, to_move)        

In [ ]:
copy_files_with_statuto("../atti_costitutivi/", True)

1 5056147070001.pdf True
2 4996401920001.pdf False
3 5010294970001.pdf True
4 4935800480001.pdf False
5 4931841280001.pdf False
6 5085888340001.pdf False
7 5037982010001.pdf False
8 5029573920001.pdf True
9 5081129070001.pdf True
10 4985002750001.pdf True
11 4937863090001.pdf True
12 4999270280001.pdf False
13 5119287680001.pdf True
14 4990994940001.pdf False
15 4972703470001.pdf False
16 4918990710001.pdf False
17 5000138680001.pdf True
18 4954928220001.pdf False
19 5087066450001.pdf False
20 4961068670001.pdf False
21 4972212500001.pdf False
22 5022334380001.pdf False
23 5051117410001.pdf True
24 5068004440001.pdf False
25 5023252310001.pdf False
26 4920791510001.pdf False
27 5025938630001.pdf False
28 5018365490001.pdf False
29 5095490550001.pdf False
30 5085792750001.pdf True
31 5043180750001.pdf False
32 5000700550001.pdf True
33 5058168270001.pdf False
34 5068910870001.pdf False
35 5009468040001.pdf False
36 5039851510001.pdf True
37 4931827850001.pdf False
38 5042829680001.pdf T

### ToDo: Gestire 4926305720001.pdf e 4909480860001.pdf